# Model Training
Trains a model on past data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

### SET PARAMETERS:


In [ ]:
TICKER="TDOC"
DATA_INTERVAL_MINUTES = 15   # (Set to 5 or 15)
DATA_AFTER_HOURS = False

DAYS_PREDICT = 3
# ISSUE: If stock goes down slowly, less than 'DOWN_PCTS_PREDICT' then won't sell but after few periods will be very down 
#  example: DOWN_PCTS_PREDICT=5% then down 4% and down 3% and down 1%... and never sell
DOWN_PCTS_PREDICT= [3.0]
UP_PCTS_PREDICT= [3.0]

signal_avg= [
    2, 
    3, 
    5, 
    8, 
    13, 
    21, 
    34, 
    55, 
    89, 
    144, 
    233, 
    377, 
    610, 
    987, 
    1597, 
    2584]

PREDICT_UP = False
if PREDICT_UP:
    INDEX_KEEP= 2
    INDEX_REMOVE_A= 0
    INDEX_REMOVE_B= 1
else:
    INDEX_KEEP= 0
    INDEX_REMOVE_A= 1
    INDEX_REMOVE_B= 2

TRAIN_SPLIT = 0.9

# TODO: When executing only using 33-38% GPU - Try different BATCH_SIZE see if parallelism increases? Learning decreases because less batches?
BATCH_SIZE= 32

HIDDEN_UNITS=12

TRAINING_THRESHOLD = 0.7

#### DOWNLOAD DATA (DON'T EXECUTE IF ALREADY LOADED)

In [ ]:
import sys
sys.path.append('..\\..')

from datetime import datetime

import settings
import apis.tiingo_api as tiingo

secret_key= settings.get_secret("tiingo-key")

### TRAINING DATA
csv_data2017= tiingo.download_ticker(secret_key, TICKER, datetime(2017, 1, 1), datetime(2018,1,1), DATA_INTERVAL_MINUTES, DATA_AFTER_HOURS)
csv_data2018= tiingo.download_ticker(secret_key, TICKER, datetime(2018, 1, 1), datetime(2019,1,1), DATA_INTERVAL_MINUTES, DATA_AFTER_HOURS)
csv_data2019= tiingo.download_ticker(secret_key, TICKER, datetime(2019, 1, 1), datetime(2020,1,1), DATA_INTERVAL_MINUTES, DATA_AFTER_HOURS)
csv_data2020= tiingo.download_ticker(secret_key, TICKER, datetime(2020, 1, 1), datetime(2021,1,1), DATA_INTERVAL_MINUTES, DATA_AFTER_HOURS)
csv_data2021= tiingo.download_ticker(secret_key, TICKER, datetime(2021, 1, 1), datetime(2022,1,1), DATA_INTERVAL_MINUTES, DATA_AFTER_HOURS)
csv_data2022= tiingo.download_ticker(secret_key, TICKER, datetime(2022, 1, 1), datetime(2023,1,1), DATA_INTERVAL_MINUTES, DATA_AFTER_HOURS)
csv_data2023= tiingo.download_ticker(secret_key, TICKER, datetime(2023, 1, 1), datetime(2024,1,1), DATA_INTERVAL_MINUTES, DATA_AFTER_HOURS)
# csv_data2024= tiingo.download_ticker(secret_key, TICKER, datetime(2024, 1, 1), datetime(2024,2,1), DATA_INTERVAL_MINUTES, DATA_AFTER_HOURS)

In [ ]:
import io
import pandas as pd

# ### TRAINING DATA
df2017 = pd.read_csv(io.StringIO(csv_data2017))
df2018 = pd.read_csv(io.StringIO(csv_data2018))
df2019 = pd.read_csv(io.StringIO(csv_data2019))
df2020 = pd.read_csv(io.StringIO(csv_data2020))
df2021 = pd.read_csv(io.StringIO(csv_data2021))
df2022 = pd.read_csv(io.StringIO(csv_data2022))
df2023 = pd.read_csv(io.StringIO(csv_data2023))
# df2024 = pd.read_csv(io.StringIO(csv_data2024))

# if not df2017.empty:
#     print("Concatenating from 2017")
#     df = pd.concat([df2017, df2018, df2019, df2020, df2021, df2022, df2023], axis=0, ignore_index=True)
# elif not df2018.empty:
#     print("Concatenating from 2018")
#     df = pd.concat([df2018, df2019, df2020, df2021, df2022, df2023], axis=0, ignore_index=True)
# el
if not df2019.empty:
    print("Concatenating from 2019")
    df = pd.concat([df2019, df2020, df2021, df2022, df2023], axis=0, ignore_index=True)
elif not df2020.empty:
    print("Concatenating from 2020")
    df = pd.concat([df2020, df2021, df2022, df2023], axis=0, ignore_index=True)
else:
    print("Concatenating from 2021")
    df = pd.concat([df2021, df2022, df2023], axis=0, ignore_index=True)

# if not df2024.empty:
#     print("Concatenating from 2024")
#     df = pd.concat([df, df2024], axis=0, ignore_index=True)

In [ ]:
# Validates that data has been concatenated correctly = ordered ascending
if df["date"].is_monotonic_increasing and df["date"].is_unique:
    print("Correct: DataFrame is in ascending order.")
else:
    print("Error: DataFrame is not in ascending order.")


In [ ]:
# Displays first and last element in the data
print(f"Data first:\n{df[['date', 'close']][:5]}")
print(f"Data last:\n{df[['date', 'close']][-5:]}")

In [ ]:

# If quotes every 15min there 26 per day if quotes every 5min there are 78 per day
TICKS_IN_DAY = 26 if DATA_INTERVAL_MINUTES == 15 else 78
# How many data ticks are inspecting to determine the if up or down by percentage 
TICKS_PREDICT= TICKS_IN_DAY * DAYS_PREDICT
REACH_PCT= 0.95

import classifiers.up_down_classifier as udc
import classifiers.ewa_classifier as ec

alpha= ec.calculate_ewa_alpha(TICKS_PREDICT, REACH_PCT)
print(f"alpha: {alpha:.4f} for window: {TICKS_PREDICT} and reach: {REACH_PCT}")

classes_calc = udc.UpsDownsClassifier(TICKS_PREDICT, DOWN_PCTS_PREDICT, UP_PCTS_PREDICT)

close_prices = df['close'].astype(float).tolist()
input_data= ec.calculate_ewas(close_prices, alpha)

classes= classes_calc.classify(input_data)
print(f"Check correct 'nan' point (window={TICKS_PREDICT}): {classes[-TICKS_PREDICT-1:-TICKS_PREDICT+1]}")
print(f"prices vs input_data: {[(p, c) for p, c in zip(close_prices[2650:3000], input_data[2650:3000])]}")
print(f"input_data vs classes: {[(p, c) for p, c in zip(input_data[2650:3000], classes[2650:3000])]}")


In [ ]:
import matplotlib.pyplot as plt

# Create a histogram
hist_values, bin_edges, _ = plt.hist(classes, bins=3, edgecolor='black')

# Add labels and a title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Data')

# Display frequency on top of each bar
for value, edge in zip(hist_values, bin_edges[:-1]):
    plt.text(float(edge), float(value), str(int(value)), color='black')
    
# Show the histogram
plt.show()


In [ ]:
# Show percentages of each class value
import utils.list_utils as lu

lu.display_frequency_classes(classes, DOWN_PCTS_PREDICT, UP_PCTS_PREDICT)

In [ ]:
# Display classes value changes over time (last 500 ticks)
graph_ticks = 500
x = range(len(classes[-graph_ticks:]))

plt.figure(figsize=(20,5))
plt.plot(x, classes[-graph_ticks:], linestyle='-')


plt.xlabel('Index')
plt.ylabel('Class')
plt.title('Plot of Classes')

plt.show()

In [ ]:

# Calculate the signals as input for the neural network as proportions
import preprocessing.proportions_calc as proportions

signals_calculator = proportions.ProportionsCalc(signal_avg)

proportions_avg = signals_calculator.calculate(close_prices)


In [ ]:
print(f"Prices length: {len(close_prices)}")
print(f"Proportions length: {len(proportions_avg[-1])}")

print(f"Last 10 close: {close_prices[-10:]}")
print(f"Last 10 proportions(avg={signal_avg[0]}): {proportions_avg[0][-10:]}")

print(f"Proportions avgs: Count: {len(signal_avg)} Max: {signal_avg[-1]}")
# At the end of the data, when less ticks than necessary no possible to predict so "nan" 
print(f"Classes last non-nan: {classes[-TICKS_PREDICT-1:-TICKS_PREDICT+1]} len: {len(classes)}")
print(f"Proportions first non-nan(avg={signal_avg[-1]}): {proportions_avg[-1][signal_avg[-1]-2:signal_avg[-1]]} len: {len(proportions_avg[-1])}")
print(f"Proportions (avg={signal_avg[0]}) Min: {min(proportions_avg[0][signal_avg[0]-1:-TICKS_PREDICT-1])} Max: {max(proportions_avg[0][signal_avg[0]-1:-TICKS_PREDICT])}")
print(f"Proportions (avg={signal_avg[-1]}) Min: {min(proportions_avg[-1][signal_avg[-1]-1:-TICKS_PREDICT-1])} Max: {max(proportions_avg[-1][signal_avg[-1]-1:-TICKS_PREDICT])}")

In [ ]:
# Removing the "nan" from the proportions
#   At the beging first signal_avg[-1] are "nan" (need previous values for first avg.)
#   At the end decided not predict if period to predict is shorter
targets = classes[signal_avg[-1]-1:-TICKS_PREDICT]
inputs = []
for proportion in proportions_avg:
    proportion_cut= proportion[signal_avg[-1]-1:-TICKS_PREDICT]
    print(proportion_cut[:2])
    inputs.append(proportion_cut)

print(f"First target: {targets[0]} and last target: {targets[-1]}")
print(f"Classes: {len(classes)} after cut to targets: {len(targets)}")
print(f"Inputs len: {len(inputs[len(signal_avg)-1])}")
print(f"Distinct targets: {list(set(targets))}")


In [ ]:
from collections import Counter

print("Train data: 'nan' removed from begining and end")
lu.display_frequency_classes(targets, DOWN_PCTS_PREDICT, UP_PCTS_PREDICT)

targets_frequency = Counter(targets)
print("VALIDATE removing should be POSITIVE?")
count_remove_a= targets_frequency[INDEX_REMOVE_A] - targets_frequency[INDEX_KEEP] + targets_frequency[INDEX_KEEP] //2
count_remove_b= targets_frequency[INDEX_REMOVE_B] - targets_frequency[INDEX_KEEP] + targets_frequency[INDEX_KEEP] //2
print(f"Removing {INDEX_REMOVE_A}: {count_remove_a}")
print(f"Removing {INDEX_REMOVE_B}: {count_remove_b}")


In [ ]:
###################
##### SET PARAMETERS
###################
# DECISION SET: REMOVING?
indexes_remove_a= []
# 2024-03-01 Do not remove anything
# if count_remove_a > 0:
#     indexes_remove_a = get_indexes_value(targets, index_remove_a, count_remove_a)

# DECISION SET: REMOVING?
indexes_remove_b= []
# 2024-03-01 Do not remove anything
# if count_remove_b > 0:
#     indexes_remove_b = get_indexes_value(targets, index_remove_b, count_remove_b)

indexes_remove= indexes_remove_a + indexes_remove_b
targets_clean= lu.remove_indexes(targets, indexes_remove)

lu.display_frequency_classes(targets_clean, DOWN_PCTS_PREDICT, UP_PCTS_PREDICT)
print(f"Targets len: {len(targets)} Targets clean: {len(targets_clean)} Difference: {len(targets)-len(targets_clean)}")

inputs_clean = [lu.remove_indexes(input, indexes_remove) for input in inputs]    
print(f"targets_clean positions(Keep={INDEX_KEEP})(First:{targets_clean.index(INDEX_KEEP)},Last:-{targets_clean[::-1].index(INDEX_KEEP)})")


In [ ]:
# Sets 'index_keep' as target = 1 and rest of indexes to target=0
targets_binary= lu.convert_binary(targets_clean, INDEX_KEEP)
print(f"targets_binary First {targets_binary.index(True)} and Last(counting from end) {targets_binary[::-1].index(True)} position with True")
print(f"targets_binary len: {len(targets_binary)} Input clean[0]: {len(inputs_clean[0])} Input clean[-1]: {len(inputs_clean[-1])}")

In [ ]:
import torch

inputs_tensor = torch.Tensor(inputs_clean)
print(f"inputs_tensor: {inputs_tensor.size()}")
inputs_tensor = inputs_tensor.T
print(f"inputs_tensor: {inputs_tensor.size()}")
targets_tensor = torch.Tensor(targets_binary)
print(f"inputs_clean len0 x len1: {len(inputs_clean)} x {len(inputs_clean[0])} -> inputs_tensor.shape: {inputs_tensor.shape}")
print(f"targets_binary.shape: {len(targets_binary)} -> targets_tensor.shape: {targets_tensor.shape}")
print(f"inputs_tensor: {inputs_tensor}")
print(f"targets_tensor: {targets_tensor}")

In [ ]:
#Shuffle tensors
torch.manual_seed(42) 
permutation = torch.randperm(inputs_tensor.size(0))
inputs_tensor_shuffle = inputs_tensor[permutation]

targets_tensor_shuffle = targets_tensor[permutation]

print(f"inputs_tensor.size(0): {inputs_tensor.size(0)}")
print(f"inputs_tensor.shape: {inputs_tensor.shape} -> inputs_tensor_shuffle.shape: {inputs_tensor_shuffle.shape}")
print(f"targets_tensor.shape: {targets_tensor.shape} -> targets_tensor_shuffle.shape: {targets_tensor_shuffle.shape}")

In [ ]:

train_test_split= int(TRAIN_SPLIT * len(targets_tensor))
inputs_tensor_train, targets_tensor_train= inputs_tensor_shuffle[:train_test_split], targets_tensor_shuffle[:train_test_split]
inputs_tensor_test, targets_tensor_test= inputs_tensor_shuffle[train_test_split:], targets_tensor_shuffle[train_test_split:]


In [ ]:
print("Training dataset frequencies:")
lu.display_frequency_values(targets_tensor_train.tolist())
print("Validation dataset frequencies:")
lu.display_frequency_values(targets_tensor_test.tolist())


In [ ]:
from typing import Tuple
import torch
from torch.utils.data import Dataset

class StockDataset(Dataset):
    def __init__(
        self,
        inputs,
        targets):
        
        self.inputs= inputs
        self.targets= targets

    def __len__(self) -> int:
        return len(self.targets)

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        return self.inputs[index], self.targets[index]


In [ ]:
from torch.utils.data import DataLoader


train_dataset= StockDataset(
  inputs_tensor_train,
  targets_tensor_train
)

print(f"train_dataset: {train_dataset[0]}")

train_dataloader= DataLoader(
  dataset=train_dataset,
  batch_size=BATCH_SIZE,
  shuffle=False
)

train_input0, train_target0= next(iter(train_dataloader))
print(f"train_input0: {train_input0} train_target0: {train_target0}")


In [ ]:
test_dataset= StockDataset(
  inputs_tensor_test,
  targets_tensor_test
)

print(f"train_dataset: {train_dataset[0]}")

val_dataloader= DataLoader(
  dataset=test_dataset,
  batch_size=BATCH_SIZE,
  shuffle=False
)

test_input0, test_target0= next(iter(val_dataloader))
print(f"test_input0: {train_input0} test_target0: {test_target0}")


In [ ]:
# EXECUTE FROM THIS STEP To CREATE A NETWORK WITH RANDOM WEIGHTS

import torch
from torch import nn

class StockModelBinaryV0(nn.Module):
  def __init__(self, input_features, hidden_units):
    """Initializes multi-class classification model"""
    super().__init__()
    self.linear_layer_stack = nn.Sequential(
      nn.Linear(in_features=input_features, out_features=hidden_units*16),
      nn.LeakyReLU(negative_slope=0.1),
      nn.Linear(in_features=hidden_units*16, out_features=hidden_units*8),
      nn.LeakyReLU(negative_slope=0.1),
      nn.Linear(in_features=hidden_units*8, out_features=hidden_units*4),
      nn.LeakyReLU(negative_slope=0.1),
      nn.Linear(in_features=hidden_units*4, out_features=hidden_units),
      nn.LeakyReLU(negative_slope=0.1),
      nn.Linear(in_features=hidden_units, out_features=1)
    )

  def forward(self, x):
    # print("forward x: ",", ".join([str(num) for num in x.tolist()]))
    # Layers are defined inside the Sequencial NN and will be applied here.
    return self.linear_layer_stack(x)

# Create an instance of the model
model_0 = StockModelBinaryV0(
  input_features=len(signal_avg),
  hidden_units=HIDDEN_UNITS).to(device)


In [ ]:
import torchmetrics

# loss_fn = nn.BCEWithLogitsLoss()

# Loss function for an imbalanced dataset (there many more 0's than 1's). Apply more weight to the less frequent class
num_ones = torch.count_nonzero(targets_tensor_train)
num_zeros = len(targets_tensor_train)-num_ones
pos_weight = num_zeros / num_ones
print(pos_weight)
# pos_weight_tensor = torch.tensor([pos_weight]).to(device)
# pos_weight_tensor = torch.tensor([1.5]).to(device)
print(f"Train negative: {num_zeros} positive: {num_ones} pos_weight: {pos_weight}")

loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


In [ ]:
###################
##### SET PARAMETERS
###################
# PERFORMANCE_MEASURE="accu"
# performance_fn= torchmetrics.Accuracy(task='binary').to(device)
PERFORMANCE_MEASURE="prec"
performance_fn= torchmetrics.Precision(task='binary').to(device)
# PERFORMANCE_MEASURE="reca"
# performance_fn= torchmetrics.Recall(task='binary').to(device)
###################
##### SET PARAMETERS
###################

In [ ]:
# Should you reset BEST PERFORMANCE
best_val_performance = 0
model_best = None

In [ ]:
###################
##### SET PARAMETERS
###################
# DO: AFTER THIS CELL RUNS EXECUTE CELLS UNTIL SAVE STEP TO KEEP BEST RESULT IN CASE IT GOES DOWN
#    lr = 0.1 -> 0.03 -> 0.001
#    epochs 200 + 200 (lr=0.1) -> 100 (lr=0.03) -> 100 (lr=0.001)
# EXECUTE 0.1 x 200 for 2 TIMES (or 400 for 1 time)
# optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.1)
# epochs=400
# optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.03)
# epochs=100
# optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.005)
# epochs=100
# optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.0007)
# epochs=100
# optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.0001)
# epochs=100
###Using Adam optimizer
# learning_rate = 0.005
# beta_1 = 0.9
# beta_2 = 0.999
# decay = 0.01
# optimizer = optim.Adam(params=model_0.parameters(), lr=learning_rate, betas=(beta_1, beta_2), eps=1e-8, weight_decay=decay)
# epochs=2000

EPOCHS=1000
LEARNING_RATE= 0.1
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=LEARNING_RATE)

##### Using a StepLR Scheduler
# from torch.optim.lr_scheduler import StepLR
# gamma = 0.95
# scheduler = StepLR(optimizer, step_size=20, gamma=gamma)
## If continue optimization
# learning_rate_last= 0.0080995 * 0.95
# gamma = 0.95
# optimizer = torch.optim.SGD(params=model_0.parameters(), lr=learning_rate_last*gamma)
# scheduler = StepLR(optimizer, step_size=20, gamma=gamma)
# epochs= 200

##### Using a ReduceLROnPlateau
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
###################
##### SET PARAMETERS
###################

In [ ]:
from copy import deepcopy

EARLY_STOPPING_PATIENCE = 60

best_model_keep = "high_prec"
best_val_loss = float('inf')
early_stopping_counter = 0

for epoch in range(EPOCHS):
    train_loss, train_performance= 0, 0
    train_samples = 0

    # Training
    model_0.train()

    for batch, (X, y) in enumerate(train_dataloader):
        X= X.to(device)
        y= y.to(device)

        # Forward pass
        y_logits= model_0(X).view(-1)
        
        # turn logits -> prediction probabilities -> prediction labels
        y_sigmoid_output = torch.sigmoid(y_logits)
        y_pred = (y_sigmoid_output > TRAINING_THRESHOLD).float()
        
        # Calculate loss and accuracy
        loss= loss_fn(y_logits, y)
        train_loss+= loss * X.size(0)
        train_performance+= performance_fn(y_pred, y) * X.size(0)
        train_samples += X.size(0)
        
        # Zero the gradients to avoid accomulating gradients from previous iteration
        optimizer.zero_grad()
        
        # Backpropagation
        loss.backward()
        
        # Updates the model usign the gradients
        optimizer.step()
    
    train_loss /= train_samples
    train_performance /= train_samples
      
    model_0.eval()
    val_loss, val_performance= 0, 0
    val_samples = 0
    with torch.inference_mode():
        for X, y in val_dataloader:
            X= X.to(device)
            y= y.to(device)
        
            # Predict for test data
            val_logits= model_0(X).view(-1)
            sigmoid_output = torch.sigmoid(val_logits)
            val_pred = (sigmoid_output > TRAINING_THRESHOLD).float()
            
            # Calculate test loss/accuracy
            val_loss+= loss_fn(val_logits, y) * X.size(0)
            val_performance+= performance_fn(val_pred, y) * X.size(0)
            val_samples += X.size(0)

        val_loss /= val_samples
        val_performance /= val_samples
    
    # Update the learning rate?
    if scheduler.__class__.__name__ == "ReduceLROnPlateau":
        scheduler.step(val_loss)
    else:
        scheduler.step()

    if epoch % 10 == 0:
       print(f"Epoch: {epoch+1} lr: {optimizer.param_groups[0]['lr']:.7f} | Loss: {train_loss:.5f} {PERFORMANCE_MEASURE}: {train_performance*100:.4f}% | Val loss: {val_loss:.5f} Val {PERFORMANCE_MEASURE}: {val_performance*100:.4f}%")

    ## Store model with higher precision
    if best_model_keep == "high_prec" and val_performance > best_val_performance:
        best_val_performance = val_performance
        model_best = deepcopy(model_0)
            
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        #Store model with lower validation loss?
        if best_model_keep == "lower_loss":
            model_best = deepcopy(model_0)
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= EARLY_STOPPING_PATIENCE:
            print("Early stopping triggered.")
            break    

print(f"Finished: Epoch: {epoch+1} lr: {optimizer.param_groups[0]['lr']:.7f} | Loss: {train_loss:.5f} {PERFORMANCE_MEASURE}: {train_performance*100:.4f}% | Val loss: {val_loss:.5f} Val {PERFORMANCE_MEASURE}: {val_performance*100:.4f}%")

In [ ]:
from torchmetrics import ConfusionMatrix, Accuracy, Precision
from mlxtend.plotting import plot_confusion_matrix

model_best.eval()
val_loss, val_performance= 0, 0
with torch.inference_mode():
    X= inputs_tensor_test.to(device)
    y= targets_tensor_test.to(device)

    # Predict for validation data
    val_logits= model_best(X).view(-1)
    sigmoid_output = torch.sigmoid(val_logits)
    val_pred = (sigmoid_output > TRAINING_THRESHOLD).float()
    
    # Calculate loss/performance(accuracy|precision)
    val_loss+= loss_fn(val_logits, y)
    val_performance+= performance_fn(val_pred, y)

print(f"Validation loss: {val_loss:.5f} Performance {PERFORMANCE_MEASURE}: {val_performance*100:.7f}%")

confmat= ConfusionMatrix(task='binary')

# test_data.targets are the values we want to predict in the test dataloader
confmat_tensor= confmat(
  preds= val_pred.cpu(),
  target= targets_tensor_test.cpu())

# Plot confusion matrix
fig, ax= plot_confusion_matrix(
  conf_mat= confmat_tensor.numpy(),
  figsize= (10, 7)
)

accuracy_fn= Accuracy(task='binary').to(device)
val_accuracy = accuracy_fn(val_pred, y)
print(f"Train validation confusion matrix:\n{confmat_tensor}")

precision_fn= Precision(task='binary').to(device)
val_precision = precision_fn(val_pred, y)
print(f"Train validation accuracy: {val_accuracy*100:.2f}%")
print(f"Train validation precision: {val_precision*100:.2f}%")
false_positives = confmat_tensor[0, 1].item()
true_positives = confmat_tensor[1, 1].item()
print(f"Train validation false_positives: {false_positives} true_positives: {true_positives}")

#### Summary

In [ ]:
import utils.text_utils as tu

date_start= df['date'][0]
date_end= df['date'].iloc[-1]

target= "UP" if PREDICT_UP else "DOWN"
MODEL_NAME= f"{datetime.now().strftime('%Y-%m-%d-%H%M')}-{TICKER}-predict{target}-dates{tu.shorten_date(date_start)}-{tu.shorten_date(date_end)}-days{DAYS_PREDICT}-down{int(DOWN_PCTS_PREDICT[0]*100)}-up{int(UP_PCTS_PREDICT[0]*100)}-in{len(signal_avg)}-hid{HIDDEN_UNITS}-pos_weight{pos_weight*10000:.0f}-{PERFORMANCE_MEASURE}{val_performance*10000:.0f}pct-fp{false_positives}tp{true_positives}-{best_model_keep}.pth"

print("======TRAINING:")
print(f"Ticker: {TICKER}")

print("--Data")
print(f"Start: {date_start} End: {date_end}")
print(f"Interval: {DATA_INTERVAL_MINUTES} - After Hours: {DATA_AFTER_HOURS}")
print("Targets Frequencies:")
lu.display_frequency_classes(targets_clean, DOWN_PCTS_PREDICT, UP_PCTS_PREDICT)

print(f"Signal Averages: {signal_avg}")
print(f"Train predict {'UP' if PREDICT_UP else 'DOWN'} - Days: {DAYS_PREDICT} Down pcts: {DOWN_PCTS_PREDICT} Up pcts: {UP_PCTS_PREDICT}")

print("Training dataset frequencies:")
lu.display_frequency_values(targets_tensor_train.tolist())
print("Validation dataset frequencies:")
lu.display_frequency_values(targets_tensor_test.tolist())

print("--Training")
print(f"Network hidden units: {HIDDEN_UNITS}")
print(f"Loss func: {loss_fn.__class__.__name__}")
print(f"Train negative: {num_zeros} positive: {num_ones} pos_weight: {pos_weight}")
print(f"Optimizer: {optimizer.__class__.__name__} Scheduler: {scheduler.__class__.__name__}")
print(f"Train/Val split: {TRAIN_SPLIT}")
print(f"Train threshold: {TRAINING_THRESHOLD}")
print(f"Epochs: {EPOCHS} learning_rate: {LEARNING_RATE}")

print("--Validation Results")
print(f"Trained model: {MODEL_NAME}")
print(f"Validation loss: {val_loss:.5f} Performance {PERFORMANCE_MEASURE}: {val_performance*100:.2f}%")
print(f"Train validation confusion matrix:\n{confmat_tensor}")
print(f"Train validation accuracy: {val_accuracy*100:.2f}%")
print(f"Train validation precision: {val_precision*100:.2f}%")
print(f"Train validation false_positives: {false_positives} true_positives: {true_positives}")


In [ ]:
# Saves model.state_dic() with best performance to a file

from pathlib import Path

# Create directory, if it doesn't exist, to store models
MODEL_PATH= Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# Create path to the model
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# torch.save(
#   obj=model_0.state_dict(),
#   f=f"{MODEL_SAVE_PATH}.pth")

torch.save(
    obj=model_best.state_dict(), 
    f=f"{MODEL_SAVE_PATH}")


print(f"Trained model saved: {MODEL_NAME}")

In [ ]:
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo

from torchinfo import summary  


In [ ]:
summary(model_0, input_size=[len(signal_avg)])